Implementing Named Entity Recognition (NER) with BERT using Feature Engineered Corpus annotated with IOB and POS tags

Link to the data:
https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 2

In [ ]:
ls -ltra

total 14868
drwxr-xr-x 4 root root     4096 Apr 19 13:29 .config/
drwxr-xr-x 1 root root     4096 Apr 19 13:30 sample_data/
drwxr-xr-x 1 root root     4096 Apr 23 09:23 ../
drwxr-xr-x 1 root root     4096 Apr 23 09:30 ./
-rw-r--r-- 1 root root 15208151 Apr 23 09:31 ner_dataset.csv


In [ ]:
import pandas as pd
df = pd.read_csv('ner_dataset.csv', encoding = 'latin1')

In [ ]:
df.head()


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
df1 = df.fillna(method = 'ffill')

In [ ]:
df1.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
df1["Sentence #"] = LabelEncoder().fit_transform(df1["Sentence #"])

In [ ]:
df1.head()

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [ ]:
df1.rename(columns = {"Sentence #": "sentence_id", "Word": "words", "Tag": "labels"}, inplace = True)

In [ ]:
df1["labels"] = df1["labels"].str.upper()

In [ ]:
X = df1[["sentence_id", "words"]]
Y = df1["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
train_data = pd.DataFrame({"sentence_id": x_train['sentence_id'], "words": x_train['words'], "labels": y_train})
test_data = pd.DataFrame({"sentence_id": x_test['sentence_id'], "words": x_test['words'], "labels": y_test})

In [ ]:
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
label = df1["labels"].unique().tolist()

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased', labels = label, args = args, use_cuda = False)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1461 [00:00<?, ?it/s]

In [ ]:
result

In [ ]:
prediction, model_output = model.predict([''])